In [1]:
import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import warnings
#import unicodedata
warnings.filterwarnings("ignore")
#from io import BytesIO
#import gzip
import requests

pd.set_option('display.max_rows',999)

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## El indicador de "% de firmas que opinan que el principal problema es la falta de capacidad 

In [3]:
dict_es = pd.read_excel('files/ES.xlsx' , sheet_name=None)

In [4]:
for key, value in dict_es.items():
    print(key)

Biggest Obstacle
Corruption
Crime
Finance
Firm Characteristics
Gender
Informality
Infrastructure
Innovation and Technology
Performance
Regulations and Taxes
Trade
Workforce


In [5]:
es = pd.DataFrame(dict_es['Biggest Obstacle'])

In [6]:
es.drop(columns=['Average/SE/N'],inplace=True)

In [7]:
es = es.replace({'Brazil':'Brasil'})

In [8]:
es.set_index(['Economy' , 'Year' , 'Subgroup' , 'Top Subgroup Level' , 'Subgroup Level'],inplace=True)

In [9]:
es.drop(es.columns[:8],axis=1,inplace=True)
es.drop(es.columns[1:],axis=1,inplace=True)

In [10]:
#Creo una lista con nombres de países de LATAM para filtrar la base es

cwpops = pd.read_excel('files/cwpops.xlsx', index_col = [0,1])
cwpops.reset_index(inplace=True)
n = list(cwpops['Country'].unique())

distances = pd.read_excel('files/distances.xlsx')
n = n + list(distances['Country'].unique())
n = list(set(n))

In [11]:
#n es una lista de 20 países de latam (Borré el código de dónde salieron)
es.reset_index(inplace=True)
es_latam = es.loc[es['Economy'].isin(n),]

In [12]:
es_latam['Year'] = es_latam['Year'].astype('Int64')

In [13]:
indices = list(es_latam[['Year' , 'Subgroup Level']].loc[(es_latam['Subgroup'] == 'Location')].reset_index().groupby('Subgroup Level').max('Year')['index'])

In [14]:
es_latam_un = es_latam.loc[indices,]

es_latam_un.rename(columns={"Economy": "Country", "Subgroup Level": "City"},inplace=True)

es_latam_un.set_index(['Country' , 'City'], inplace=True)

In [15]:
es_latam_un.drop(['Subgroup' , 'Top Subgroup Level' ], axis=1,inplace=True)

In [16]:
es_latam_un.isna().value_counts()

Year   Percent of firms choosing inadequately educated workforce as their biggest obstacle
False  False                                                                                  74
dtype: int64

In [20]:
es_latam_un = es_latam_un.sort_index()

## Comienza el trabajo con la base EJD que tiene info de género, educación de la población, etc

In [21]:
ejd = pd.read_excel('files/analisis_ejd.xlsx',  sheet_name='Sheet1 (2)')

In [22]:
ejd.rename(columns={'Pais': 'Country' , 'Ciudad':'City'},inplace=True)
ejd.set_index(['Country' , 'City'],inplace=True)

In [23]:
sel = ejd.loc[:,('informal' , 'formal' , 'hombre' , 'mujer' , 'nivedu1' , 'nivedu2' , 'nivedu3')]

In [24]:
#Creo DICT para poder cambiar las siglas de los paises por los nomrbes de los paises

a = list(sel.reset_index().loc[:,'Country'].unique())

b = [
 'Brasil',
 'Costa Rica',
 'Argentina',
 'Paraguay',
 'Uruguay',
 'Mexico',
 'Peru',
 'Ecuador',
 'Bolivia',
 'Panama',
 'Venezuela',
 'Chile',
 'Colombia']

b = sorted(b)

nombres = dict(zip(a, b))

nombres['PER'] = 'Peru'
nombres['PRY'] = 'Paraguay'

In [25]:
sel = sel.reset_index().replace({"Country": nombres}).set_index(['Country','City'])

In [26]:
sel.isna().value_counts()

informal  formal  hombre  mujer  nivedu1  nivedu2  nivedu3
False     False   False   False  False    False    False      279
dtype: int64

## 'tabla_ancha.xlsx' es el archivo que posee info de empleo por sectores y salarios

Reever si algun indicador que pueda utilizar

## Hago join de las tablas para ver cuales tengo

In [27]:
joined_df = es_latam_un.join(sel, how='outer').dropna()

In [152]:
joined_df = es_latam_un.join(sel, how='inner')

In [37]:
sel.shape

(279, 7)

In [38]:
es_latam_un.reset_index()

,Country,City,Year,Percent of firms choosing inadequately educated workforce as their biggest obstacle
0,Argentina,Buenos Aires,2017,7.5
1,Argentina,Chaco,2010,6.7
2,Argentina,Cordoba,2017,2.4
3,Argentina,Mendoza,2017,2.7
4,Argentina,Rosario,2017,2.2
5,Argentina,Tucuman,2017,2.2
6,Bolivia,Cochabamba,2017,5.2
7,Bolivia,La Paz,2017,2.9
8,Bolivia,Santa Cruz,2017,6.8
9,Brasil,Amazonas,2009,8.5


In [39]:
sel.to_csv('Entregables/metricas/trabajo_cono.csv')